### Main

Run different aspects of the code in this repository. Select which sections of code to run and which not to run by commenting out different sections.

Sections:
1. train custom VGG16 model and save to models/vgg16.h5 using transfer learning. If train set to False then it will load a previously saved model. Report validataion accuracy after training and saves the model features to data/tf_vgg16_features.npy
2. train custom Inception Resnet v2 model and save to models/resnet2.h5 using transfer learning. If train set to False then it will load a previously saved model. Report validataion accuracy after training and saves the model features to data/tf_resnet2_features.npy
3. load pretrained model from https://github.com/geifmany/cifar-vgg for CIFAR-10. Report validation accuracy and save the features from the trained model to data/cifar_vgg_features.npy
4. extract histogram of oriented gradients (HOG) features to data/hog_features.npy
5. extract flatten pixel values as feature set to data/flatten_features.npy
6. train FCNNs on each of the 5 feature sets described above (1-5). Trained models are saved.
7. create two composite feature sets by doing 1000 PCA on all 5 feature sets combined and doing PCA on only the two transfer learning feature sets (1 and 2)
8. train FCNN models with the two composite feature sets from 7 and report validation accuracies. Save the models.

In [4]:
# imports
from modules import resnet2, vgg16, cifar_vgg, utils, fcnn
from scripts import hog_features, flatten_features, composite_features
import numpy as np
import os

# create directories to save outputs
os.makedirs('models', exist_ok=True)

In [1]:
# main code

# 1) TF VGG16
model = vgg16(train=True)
pred_y = model.predict()
true_y =  model.y_test
acc = utils.calculate_accuracy(pred_y, model.y_test)
print('TF VGG16 testing accuracy: {}'.format(acc*100))
utils.save_model_features(model, 'data/tf_vgg16_features')

# 2) TF resnet2
model = resnet2(train=True)
pred_y = model.predict()
true_y =  model.y_test
acc = utils.calculate_accuracy(pred_y, model.y_test)
print('TF Resnet2 testing accuracy: {}'.format(acc*100))
utils.save_model_features(model, 'data/tf_resnet2_features')

# 3) CIFAR-VGG
model = cifar_vgg(train=False)
pred_y = model.predict()
true_y =  model.y_test
acc = utils.calculate_accuracy(pred_y, model.y_test)
print('TF Resnet2 testing accuracy: {}'.format(acc*100))
utils.save_model_features(model, 'data/cifar_vgg_features')

# 4) Extract HOG features
hog_features('data/hog_features.npy')

# 5) Extract flatten pixel features
flatten_features('data/flatten_features.npy')

# 6) Test out feature description by training a simple FCNN on it
#### TF VGG16 features
model = fcnn('data/tf_vgg16_features.npy', 'models/fcnn_tf_vgg16.h5', train=True)
pred_y = model.predict()
true_y =  model.y_test
acc = utils.calculate_accuracy(pred_y, model.y_test)
print('FCNN with TF VGG16 testing accuracy: {}'.format(acc*100))

#### TF Resnet2 features
model = fcnn('data/tf_resnet2_features.npy', 'models/fcnn_tf_resnet2.h5', train=True)
pred_y = model.predict()
true_y =  model.y_test
acc = utils.calculate_accuracy(pred_y, model.y_test)
print('FCNN with TF Resnet2 testing accuracy: {}'.format(acc*100))

#### CIFAR-VGG features
model = fcnn('data/cifar_vgg_features.npy', 'models/fcnn_cifar_vgg.h5', train=True)
pred_y = model.predict()
true_y =  model.y_test
acc = utils.calculate_accuracy(pred_y, model.y_test)
print('FCNN with CIFAR-VGG testing accuracy: {}'.format(acc*100))

#### HOG features
model = fcnn('data/hog_features.npy', 'models/fcnn_hog.h5', train=True)
pred_y = model.predict()
true_y =  model.y_test
acc = utils.calculate_accuracy(pred_y, model.y_test)
print('FCNN with HOG testing accuracy: {}'.format(acc*100))

#### Flatten pixel features
model = fcnn('data/flatten_features.npy', 'models/fcnn_flatten.h5', train=True)
pred_y = model.predict()
true_y =  model.y_test
acc = utils.calculate_accuracy(pred_y, model.y_test)
print('FCNN with flatten pixels testing accuracy: {}'.format(acc*100))

# 7) Create composite feature sets
#### Combine eall 5 feature sets and use 1000 PCA with zscore True
features_list = ['data/tf_vgg16_features.npy',
                 'data/tf_resnet2_features.npy',
                 'data/cifar_vgg_features.npy',
                 'data/hog_features.npy',
                 'data/flatten_features.npy'
]
composite_features(features_list, 'data/allFeatures_pca1000_features.npy')

#### Combine only tf resnet2 and tf vgg16
features_list = ['data/tf_vgg16_features.npy',
                 'data/tf_resnet2_features.npy',
]
composite_features(features_list, 'data/tfFeatures_features.npy', zscore=False, pca=False)

# 8) Run FCNN on the 2 composite features
model = fcnn('data/allFeatures_pca1000_features.npy', 'models/allFeatures_pca1000.h5', train=False)
pred_y = model.predict()
true_y =  model.y_test
acc = utils.calculate_accuracy(pred_y, model.y_test)
print('FCNN with all features plus pca 10000 testing accuracy: {}'.format(acc*100))

model = fcnn('data/tfFeatures_features.npy', 'models/tfFeatures.h5', train=False)
pred_y = model.predict()
true_y =  model.y_test
acc = utils.calculate_accuracy(pred_y, model.y_test)
print('FCNN with transfer learning features testing accuracy: {}'.format(acc*100))
